<a href="https://colab.research.google.com/github/sunyeul/ToyProjectLab/blob/feature%2Fwhisper-jax/Whisper/Whisper_JAX_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/openai/whisper/discussions/1277

## ライブラリーのインストール

In [ ]:
!pip install --upgrade pip -qq
!pip install --upgrade jax[cuda]  -qq
!pip install git+https://github.com/sanchit-gandhi/whisper-jax.git -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.0 MB/s eta 0:00:00


In [ ]:
import jax
jax.devices()

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]

## オディオファイルの準備

In [ ]:
!pip install -qq pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube

def download_from_youtube(id:str, output_path:str='.', filename:str='input.mp4'):
    yt = YouTube(f'https://youtu.be/{id}')

    yt.streams.filter(file_extension='mp4').get_by_resolution('720p').download(
        output_path=output_path,
        filename=filename
        )

In [ ]:
download_from_youtube(id='RjNcTBXTk4I')

In [ ]:
!sudo apt update && sudo apt install ffmpeg

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,721 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,344 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14

In [ ]:
!ffmpeg -y -i input.mp4 -q:a 0 -map a -hide_banner -loglevel error audio.mp3

In [ ]:
from IPython.display import Audio

Audio('audio.mp3')

## Whisper-JAXモデルの準備

In [ ]:
from whisper_jax import FlaxWhisperPipline
import jax.numpy as jnp

pipeline = FlaxWhisperPipline(
    checkpoint="openai/whisper-medium",
    # checkpoint="openai/whisper-large-v2",

    # half-precision for speed-up
    # dtype=jnp.float16,  # For most GPUs
    dtype=jnp.bfloat16,  # For A100 GPUs or TPUs

    batch_size=16
    )

In [ ]:
from jax.experimental.compilation_cache import compilation_cache as cc

cc.initialize_cache("./jax_cache")

## Whisper-JAXモデルの文字起こし

In [ ]:
%%time

# JIT compile the forward call - slow, but we only do once

text = pipeline(
    inputs="audio.mp3",
    language="en",
    return_timestamps=True,
    task="transcribe"
    )

There was an error while processing timestamps, we haven't found a timestamp as last token. Was WhisperTimeStampLogitsProcessor used?


CPU times: user 1min 20s, sys: 4.17 s, total: 1min 24s
Wall time: 51.6 s


In [ ]:
%%time

# used cached function thereafter - super fast!

text = pipeline(
    inputs="audio.mp3",
    language="en",
    return_timestamps=True,
    task="transcribe"
    )

There was an error while processing timestamps, we haven't found a timestamp as last token. Was WhisperTimeStampLogitsProcessor used?


CPU times: user 3.67 s, sys: 3.24 s, total: 6.91 s
Wall time: 3.31 s


In [ ]:
from pprint import pprint

pprint(text)

{'chunks': [{'text': ' Where am I? Ooh, fresh meat for the grinder. Pay him no '
                     "heed. He is cute, but he is... There's gotta be a way "
                     "out of here. There's no escape. The only hope is the "
                     "sweet relief of death. Whoa! Oh, you've got to be "
                     'kidding me!',
             'timestamp': (25.0, 28.0)},
            {'text': ' My army! Koopas! Goombas! Whatever those things are!',
             'timestamp': (33.0, 35.0)},
            {'text': ' We will destroy the Mushroom Kingdom! Yeah!',
             'timestamp': (37.0, 38.0)},
            {'text': ' Bowser is coming.', 'timestamp': (39.0, 41.0)},
            {'text': " I'm not afraid. I'll do anything for my brother.",
             'timestamp': (41.0, 44.0)},
            {'text': " We're going to save him. You asked for it. This is fun! "
                     "Here. We. Go! Attack! Let's end this. Oh!",
             'timestamp': (44.0, None)}],
 'text': 

## Whisperとの比較

In [ ]:
!pip install -qq git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.3 MB/s eta 0:00:00


In [ ]:
import torch
import whisper

torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = whisper.load_model(
    name='medium',
    device=DEVICE
    )

100%|█████████████████████████████████████| 1.42G/1.42G [00:52<00:00, 28.9MiB/s]


In [ ]:
%%time

result = model.transcribe(
    audio='audio.mp3',
    verbose=True,
    language='en'
    )

[00:00.000 --> 00:05.000]  Where am I?
[00:05.000 --> 00:08.000]  Ooh, fresh meat for the grinder.
[00:08.000 --> 00:12.000]  Pay him no heed. He is cute, but he is...
[00:12.000 --> 00:14.000]  There's gotta be a way out of here.
[00:14.000 --> 00:15.000]  There's no escape.
[00:15.000 --> 00:18.000]  The only hope is the sweet relief of death.
[00:18.000 --> 00:22.000]  Whoa! Oh, you've got to be kidding me!
[00:24.000 --> 00:25.000]  My army!
[00:25.000 --> 00:26.000]  Koopas!
[00:27.000 --> 00:28.000]  My army!
[00:28.000 --> 00:29.000]  Koopas!
[00:29.000 --> 00:30.000]  Koombas!
[00:30.000 --> 00:32.000]  Whatever those things are!
[00:32.000 --> 00:33.000]  Aww.
[00:34.000 --> 00:36.000]  We will destroy the Mushroom Kingdom!
[00:39.000 --> 00:40.000]  Bowser is coming.
[00:40.000 --> 00:41.000]  I'm not afraid.
[00:41.000 --> 00:43.000]  I'll do anything for my brother.
[00:43.000 --> 00:45.000]  We're going to save him.
[00:47.000 --> 00:48.000]  Yes!
[00:48.000 --> 00:49.000]

In [ ]:
from pprint import pprint

pprint(result)

{'language': 'en',
 'segments': [{'avg_logprob': -0.31310184350174464,
               'compression_ratio': 1.3771428571428572,
               'end': 5.0,
               'id': 0,
               'no_speech_prob': 0.7932783365249634,
               'seek': 0,
               'start': 0.0,
               'temperature': 0.0,
               'text': ' Where am I?',
               'tokens': [50364, 2305, 669, 286, 30, 50614]},
              {'avg_logprob': -0.31310184350174464,
               'compression_ratio': 1.3771428571428572,
               'end': 8.0,
               'id': 1,
               'no_speech_prob': 0.7932783365249634,
               'seek': 0,
               'start': 5.0,
               'temperature': 0.0,
               'text': ' Ooh, fresh meat for the grinder.',
               'tokens': [50614,
                          7951,
                          11,
                          4451,
                          4615,
                          337,
                          